### <a id="TOP"></a>
<img src="https://upload.wikimedia.org/wikipedia/commons/6/60/NISAR_artist_concept.jpg" width=150/><img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/NISAR_Mission_Logo.png" width=200/> 

***

# NASA ISRO Synthetic Aperture Radar Mission
## visualizing and streaming NISAR sample products from Earthdata
This notebook requires an active earth data account and demonstrates
how to search, stream and display the NISAR GCOV sample products that are currently being hosted in the Earthdata DAAC.
If you don't have an Earthdata login account you can visit: https://urs.earthdata.nasa.gov to register. 
<br> Two sites are explored: **Madhya Pradesh, India** and **Illinois, USA**
<br> An overview of all avaliable sample products can be found here: https://nisar-docs.asf.alaska.edu/availability-overview/
<br> Date: 2026-02-02

## 0 &emsp; Getting started

### 0.1 &emsp; Import Python Modules

##### note: if earthacess is not yet installed - uncomment the line below and install: 

In [ ]:
#!pip install earthaccess

In [ ]:
import earthaccess
import os
import h5py 
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal, osr, ogr
import geopandas as gpd
import math
import requests 
import s3fs
from pyproj import CRS

### 0.2 &emsp; earthaccess

In [ ]:
auth = earthaccess.login()


#### Madhya Pradesh, India
**acquistion date: 2025-10-17**

In [ ]:
IN_results = earthaccess.search_data(short_name = 'NISAR_L2_GCOV_BETA_V1',
                                     temporal = ('2025-10-17', '2025-10-17'), 
                                     cloud_hosted = True) # temporal agrument is a tuple of (startdate, enddate)

#### Illinois, USA
**acquisition date: 2025-11-04**

In [ ]:
US_results = earthaccess.search_data(short_name = 'NISAR_L2_GCOV_BETA_V1', 
                                     temporal = ('2025-11-03', '2025-11-03'), 
                                    cloud_hosted = True) # searches for cloud-hosted data

### 1 &emsp; Opening the data 

The sample products explored in this notebook are streamed directly into memory from the NASA EarthData Cloud, instead of being downloaded to disk. The Earthdata Cloud is hosted on Amazon Web Services (AWS), and your Earthdata login credentials are used to authenticate access.

In [ ]:
auth = earthaccess.login(strategy="interactive")

* **Note:** Streaming data from the earthaccess where they are stored takes ~ 10 minutes to establish a connection to the s3 bucket and read the full dataset in. When opening the data, a buffer is set (page_buf_size) to speed up the reading process by chunks. To further reduce computation time, bounding box coordinates can be used to only read in a subset of the image. If reading and visualizing the full image is desired, change the  `subset` variable for the sites below to **False**

#### Madhya Pradesh, India

In [ ]:
subset = True # set to False if you want to read the entire frame
#subset = False
if subset == True: 
    minx = float(749722.256661103)
    miny = float(2481723.31029981)
    maxx = float(806864.5857592882)
    maxy = float(2525154.3008363205)

In [ ]:
%%time
IN_GCOV_data = earthaccess.open(IN_results)
f = IN_GCOV_data[0]
with h5py.File(f, 'r', page_buf_size = 4*1024**3) as h5_file: #
    freq = h5_file['science']['LSAR']['GCOV']['grids']['frequencyA']
    x_coords = freq['xCoordinates'][:]
    y_coords = freq['yCoordinates'][:]
    ds_epsg = freq['projection'][()]
    # point to polorizations within the h5 dataset
    hh = freq['HHHH']
    hv = freq['HVHV']
    
    if subset == True: 
    # find indices within the image that correspond to the geojson bbox
        x_idx = np.where((x_coords >= minx) & (x_coords <= maxx))[0]
        y_idx = np.where((y_coords >= miny) & (y_coords <= maxy))[0]

        xmin, xmax = x_idx.min(), x_idx.max() + 1
        ymin, ymax = y_idx.min(), y_idx.max() + 1
    # read only the subset
        ds_hh = hh[ymin:ymax, xmin:xmax]
        ds_hv = hv[ymin:ymax, xmin:xmax]
        ds_x = x_coords[xmin:xmax]
        ds_y = y_coords[ymin:ymax]  
    else:
    # reads the entire image
        ds_hh = hh[()] # this loads the entire dataset into a numpy array
        ds_hv = hv[()] # this laods the entire dataset into a numpy array
        ds_x = x_coords[()]
        ds_y = y_coords[()]
# Compute geotransform (works for both subset and full image read)
    xres = abs(ds_x[1] - ds_x[0])
    yres = abs(ds_y[1] - ds_y[0])

    ulx = ds_x[0] - xres / 2
    uly = ds_y[0] - yres / 2

    geotransform = [ulx, xres, 0.0, uly, 0.0, -yres]

    # Dimensions
    rows = ds_y.shape[0]
    cols = ds_x.shape[0]
    # clean up disk 
    del ds_x, ds_y

print(f"Subset enabled: {subset}")
print(f"Rows: {rows}, Cols: {cols}")
print("Geotransform:", geotransform)
print("EPSG:", ds_epsg)    

#### Illinois, USA

this dataset is **quad-pol**, which it is read slightly differently than the dual-pol example above

In [ ]:
subset = True # set to False if you want to read the entire frame
if subset == True: 
    minx = float(161593.730285862)
    miny = float(4422646.026707129)
    maxx = float(223798.065777231)
    maxy = float(4529409.67127771)

In [ ]:
%%time
US_GCOV_data = earthaccess.open(US_results)
f = US_GCOV_data[0]
with h5py.File(f, 'r', page_buf_size = 4*1024**3) as h5_file:
    freq = h5_file['science']['LSAR']['GCOV']['grids']['frequencyA']
    x_coords = freq['xCoordinates'][:]
    y_coords = freq['yCoordinates'][:]
    ds_epsg = freq['projection'][()]
    # point to polorizations within the h5 dataset
    hh = freq['HHHH']
    hv = freq['HVHV']
    vv = freq['VVVV']
    if subset == True: 
    # find indices within the image that correspond to the geojson bbox
        x_idx = np.where((x_coords >= minx) & (x_coords <= maxx))[0]
        y_idx = np.where((y_coords >= miny) & (y_coords <= maxy))[0]

        xmin, xmax = x_idx.min(), x_idx.max() + 1
        ymin, ymax = y_idx.min(), y_idx.max() + 1
    # read only the subset
        ds_hh = hh[ymin:ymax, xmin:xmax]
        ds_hv = hv[ymin:ymax, xmin:xmax]
        ds_vv = vv[ymin:ymax, xmin:xmax]
        ds_x = x_coords[xmin:xmax]
        ds_y = y_coords[ymin:ymax]  
    else:
    # reads the entire image
        ds_hh = hh[()] # this loads the entire dataset into a numpy array
        ds_hv = hv[()] # this laods the entire dataset into a numpy array
        ds_vv = vv[()]
        ds_x = x_coords[()]
        ds_y = y_coords[()]
# Compute geotransform (works for both subset and full image read)
    xres = abs(ds_x[1] - ds_x[0])
    yres = abs(ds_y[1] - ds_y[0])

    ulx = ds_x[0] - xres / 2
    uly = ds_y[0] - yres / 2

    geotransform = [ulx, xres, 0.0, uly, 0.0, -yres]

    # Dimensions
    rows = ds_y.shape[0]
    cols = ds_x.shape[0]
    # clean up disk 
    del ds_x, ds_y

print(f"Subset enabled: {subset}")
print(f"Rows: {rows}, Cols: {cols}")
print("Geotransform:", geotransform)
print("EPSG:", ds_epsg)    

### 1.1 &emsp; Create an RGB composite

In [ ]:
dB_min = -25
dB_max = -5

##### Madhya Pradesh, India: **Dual-Pol** <br> RGB = **R**: HH, **G**: HV, **B**: HH-HV

In [ ]:
# convert to dB for visualization
hh_dB = 10*np.log10(np.abs(ds_hh))
hv_dB = 10*np.log10(np.abs(ds_hv))
ratio_dB = hh_dB - hv_dB

In [ ]:
print(np.nanmean(hh_dB))
print(np.nanmean(hv_dB))

In [ ]:
R = np.clip((hh_dB - dB_min) / (dB_max - dB_min), 0, 1)
G = np.clip((hv_dB - dB_min) / (dB_max - dB_min), 0, 1)
B = np.clip((ratio_dB + 10) / 20, 0, 1)

In [ ]:
def stretch(x, pmin=2, pmax=98):
    lo = np.nanpercentile(x, pmin)
    hi = np.nanpercentile(x, pmax)
    return np.clip((x - lo) / (hi - lo), 0, 1)

R = stretch(R, 3, 97)
G = stretch(G, 3, 97)
B = stretch(B, 2, 98)

# Adjust channel weights slightly
wR, wG, wB = 0.73, 0.71, 0.46 # this controls the weight of each pol, i.e. if wR is set higher, HH will appear more dominant in the RGB composite
rgb = np.dstack((R*wR, G*wG, B*wB))
rgb = np.clip(rgb, 0, 1) ** 1.01  # brightens pixels for visualization purposes on a per-channel basis 

# Display
plt.figure(figsize=(7,7))
plt.title('Madhya Pradesh, India\n 2025-10-17\n R:HH, G:HV, B:HH-HV')

plt.axis('off')
plt.tight_layout()
if subset == True:
    plt.imshow(rgb)
else:
    plt.imshow(rgb[::10, ::10]) # displays every 10th pixel to save on memory and display faster


##### Illinois, USA: **Quad-Pol** <br>RGB = **R**: HH, **G**: HV, **B**: VV

In [ ]:
# convert to dB for visualization
hh_dB = (10*np.log10(np.abs(ds_hh)))
hv_dB = (10*np.log10(np.abs(ds_hv)))
vv_dB = (10*np.log10(np.abs(ds_vv)))

R = np.clip((hh_dB - dB_min) / (dB_max - dB_min), 0, 1)
G = np.clip((hv_dB - dB_min) / (dB_max - dB_min), 0, 1)
B = np.clip((vv_dB - dB_min) / (dB_max - dB_min), 0, 1)

In [ ]:
def stretch(x, pmin=2, pmax=98):
    lo = np.nanpercentile(x, pmin)
    hi = np.nanpercentile(x, pmax)
    return np.clip((x - lo) / (hi - lo), 0, 1)

R = stretch(R, 3, 97)
G = stretch(G, 3, 97)
B = stretch(B, 2, 98)

# Adjust channel weights slightly
wR, wG, wB = 0.43, 0.47, 0.44 # reduced green weight slightly to fix yellow
rgb = np.dstack((R*wR, G*wG, B*wB))
rgb = np.clip(rgb, 0, 1) ** 1.01

# Display
plt.figure(figsize=(7,7))
plt.title('Illinois, USA \n 2025-11-03 \n R:HH, G:HV, B:VV')
plt.axis('off')
plt.tight_layout()
if subset == True:
    plt.imshow(rgb)
else:
    plt.imshow(rgb[::10, ::10]) # displays every 10th pixel to save on memory and display faster